In [995]:
from pyomo.core import *
from pyomo.opt import SolverFactory, SolverManagerFactory
import pyomo.environ
import pandas as pd

from itertools import compress
import xlrd
import numpy as np
import time
start_time = time.time()

In [996]:
def Dict1D(dictVar,dataframe):
    for i,vali in enumerate(dataframe.index):
        dictVar[i+1]=dataframe.iloc[i][1]
    return dictVar

def DictND(dictVar,dataframe):
    for i,vali in enumerate(dataframe.index):
        for j,valj in enumerate(dataframe.columns):
            #dictVar[i+1,j+1]=dataframe.iloc[i][j+1]
            dictVar[vali,valj]=dataframe.iloc[i][j+1]
    return dictVar

In [986]:
#-----------------------------------------------------------------------------#
## read excel demand profil data
#-----------------------------------------------------------------------------#
path_demand = r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data/Demand.xls'
path_solar = r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data/Data.xls'
# demand book
book = xlrd.open_workbook(path_demand)
# get the first worksheet
first_sheet = book.sheet_by_index(0) 
# read a column slice
elecload_init_1 = [cell.value for cell in first_sheet.col_slice(colx=0,start_rowx=0,end_rowx=8760)]
heatload_init_1 = [cell.value for cell in first_sheet.col_slice(colx=1, start_rowx=0, end_rowx=8760)]
# solar profile
book = xlrd.open_workbook(path_solar)
# get the first worksheet
first_sheet = book.sheet_by_index(0) 
# read a column slice
solar_init_1 = [cell.value for cell in first_sheet.col_slice(colx=0,start_rowx=0,end_rowx=8760)]

elecload_init={}
for i in range(1, len(elecload_init_1)+1):  #len(elecload_init_1)+1
    elecload_init[i]=elecload_init_1[i-1]
    
heatload_init={}
for i in range(1, len(heatload_init_1)+1): #len(heatload_init_1)+1
    heatload_init[i]=heatload_init_1[i-1]
    
solar_init={}
for i in range(1, len(solar_init_1)+1): #len(heatload_init_1)+1
    solar_init[i]=solar_init_1[i-1]/1000 # W -> kW
    
loads_init={}
for i in range(1, len(elecload_init)+1):
    loads_init[i,1] = elecload_init[i]
    loads_init[i,2] = heatload_init[i]

In [987]:
excel_path=r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\General_input.xlsx'
DemandData=pd.read_excel(excel_path,sheetname='Demand data')
Elecload=DemandData["Electricity"].to_dict()
Heatload=DemandData["Heat"].to_dict()

SolarData=pd.read_excel(excel_path,sheetname='Solar data')
Irradiation=SolarData.to_dict()

Technologies=pd.read_excel(excel_path,sheetname='Technology', skiprows=1, index_col=0, skip_footer=38)
Technologies=Technologies.dropna(axis=1, how='all')
Technologies=Technologies.fillna(0)

TechOutputs=pd.read_excel(excel_path,sheetname='Technology', skiprows=15, index_col=0, skip_footer=26) #
TechOutputs=TechOutputs.dropna(axis=0,how='all')
TechOutputs=TechOutputs.dropna(axis=1,how='all')

Cost=TechOutputs.copy() #for costs

TechOutputs=TechOutputs.multiply(np.array(Technologies.loc['Efficiency']))
#TechOutputs.loc['Heat']=TechOutputs.loc['Heat'].multiply(np.array(Technologies.loc['HER'].fillna(value=1)))
TechOutputs.loc[TechOutputs.index!='Electricity']=TechOutputs.loc[TechOutputs.index!='Electricity'].multiply(np.array(Technologies.loc['HER'].fillna(value=1).replace(0,1)))
TechOutputs[TechOutputs<0]=-1

addGrid=np.zeros(len(DemandData.columns),)
addGrid[0]=1
Grid=pd.DataFrame(addGrid,columns=["Grid"],index=DemandData.columns).transpose()

##################################
######## Coupling matrix #########
##################################

addGrid=np.zeros(len(DemandData.columns),)
addGrid[0]=1
Grid=pd.DataFrame(addGrid,columns=["Grid"],index=DemandData.columns).transpose()

Cmatrix=TechOutputs.transpose()

Cmatrix=pd.concat([Grid,Cmatrix])
Cmatrix.index=list(range(1,len(TechOutputs.columns)+2))
Cmatrix.columns=list(range(1,len(TechOutputs.index)+1))
cMatrix={}
cMatrix=DictND(cMatrix,Cmatrix)

##################################
######## PART LOAD  ##############
##################################
PartLoad=Technologies.loc["MinLoad (%)",]/100

partload=Cost.iloc[0:1].mul(list(PartLoad),axis=1)
partload=pd.concat([partload,Cost.iloc[1:].mul(list(PartLoad),axis=1)], axis=0)
partload=partload.abs()
partload=partload.transpose()
partload.index=list(range(1+1,len(TechOutputs.columns)+2))
partload.columns=list(range(1,len(TechOutputs.index)+1))
SolartechsSets=list(compress(list(range(1+1,len(Technologies.columns)+2)), list(Technologies.loc["Area"]>0)))

for i in SolartechsSets:
    partload.drop(i, inplace=True)
    
    
PartloadInput={}

PartloadInput=DictND(PartloadInput,partload)

##################################
########  Max capacity  ##########
##################################
MaxCap=Technologies.loc["Maximum Capacity",]
MaxCap.index=list(range(2,len(Technologies.loc["MinLoad (%)",].index)+2))
maxCap= MaxCap.to_dict()

for i in SolartechsSets:
    maxCap.pop(i, None)

##################################
########  Linear cost  ##########
##################################
LinearCost=Technologies.loc["CapCost (chf/kW)",]

linCost=Cost.iloc[0:1].mul(list(LinearCost),axis=1)
linCost=pd.concat([linCost,Cost.iloc[1:].mul(list(LinearCost),axis=1)], axis=0)

linCost=linCost.transpose()

for name in linCost.columns[1:]:
    linCost.loc[linCost["Electricity"] >1, name] = 0

linCost.loc[linCost["Electricity"] <0, "Electricity"]=0

linCost=linCost.abs()
linCost=pd.concat([Grid,linCost])

linCost.index=list(range(1,len(TechOutputs.columns)+2))
linCost.columns=list(range(1,len(TechOutputs.index)+1))
linCost.loc[1]=0

linCapCosts={}

linCapCosts=DictND(linCapCosts,linCost)


In [988]:
Carbon=pd.read_excel(excel_path,sheetname='Technology', skiprows=24, index_col=0, skip_footer=12) #
Carbon=Carbon.dropna(axis=0,how='all')
Carbon=Carbon.dropna(axis=1,how='all')
Carbon.index=[1]

ElectricityCF=pd.read_excel(excel_path,sheetname='General', skiprows=1, index_col=0, skip_footer=10) #
ElectricityCF=ElectricityCF.dropna(axis=0,how='all')
ElectricityCF=ElectricityCF.dropna(axis=1,how='all')
del ElectricityCF["Price (chf/kWh)"]
ElectricityCF.index=[1]

CarbonFactors= pd.concat([ElectricityCF, Carbon], axis=1)
CarbonFactors.columns=list(range(1,len(CarbonFactors.columns)+1))
CarbonFactors=CarbonFactors.transpose()

carbonFactors={}
carbonFactors=Dict1D(carbonFactors,CarbonFactors)

In [955]:
carbonFactors

{1: 0.13700000000000001,
 2: 0.0,
 3: 0.19800000000000001,
 4: 0.19800000000000001,
 5: 0.0,
 6: 0.0,
 7: 0.19800000000000001,
 8: 0.19800000000000001}

In [989]:
#-----------------------------------------------------------------------------#
## Creating a model ##
#-----------------------------------------------------------------------------#
model = AbstractModel()
# number of hours, technologies, timesteps
#model.hours = Param(within=NonNegativeIntegers)
#model.hours = Param(within=NonNegativeIntegers, initilize=DemandData.shape[0])

#model.technologies = Param(within=NonNegativeIntegers,initilize=Technologies.shape[0])
#model.demand = Param(within=NonNegativeIntegers)
# sets definition
model.Time = RangeSet(1, DemandData.shape[0])
model.SubTime = RangeSet(2, DemandData.shape[0], within=model.Time)
model.In = RangeSet(1, Technologies.shape[1]+1)
model.W_Ogrid = Set(initialize=list(range(1+1,len(Technologies.columns)+2)), within=model.In)

model.Out = RangeSet(1, DemandData.shape[1])

#SolartechsSets=list(compress(list(range(1+1,len(Technologies.columns)+2)), list(Technologies.loc["Area"]>0)))
model.SolarTechs = Set(initialize=SolartechsSets, within=model.In)

DispTechsSet=list(compress(list(range(1+1,len(Technologies.columns)+2)), list(Technologies.loc["Area"]==0)))
model.DispTechs = Set(initialize=DispTechsSet, within=model.In)

model.CHP = Set(initialize=[4,7,8], within=model.In)

# coupling matrix & Technical parameters
model.cMatrix = Param(model.In, model.Out, initialize=cMatrix)                                      # coupling matrix technology efficiencies 
model.maxCapTechs = Param(model.DispTechs, initialize=maxCap)
model.maxStorCh = Param(model.Out)
model.maxStorDisch = Param(model.Out)
model.lossesStorStanding = Param(model.Out)
model.chargingEff = Param(model.Out)
model.dischLosses = Param(model.Out)
model.minSoC = Param(model.Out)
model.partLoad = Param(model.In, model.Out, initialize=PartloadInput)
model.maxSolarArea = Param(initialize=50)

# carbon factors
model.carbonFactors = Param(model.In, initialize=carbonFactors)
model.maxCarbon = Param(initialize=30000)

# Cost parameters
model.linCapCosts = Param(model.In, model.Out, initialize= linCapCosts)                                  # Technologies capital costs
model.fixCapCosts = Param(model.In, model.Out)
model.linStorageCosts = Param(model.Out)
model.opPrices = Param(model.In)                                                # Operating prices technologies
model.feedInTariffs = Param(model.Out)                                          # feed-in tariffs
model.omvCosts = Param(model.In)                                                # Maintenance costs
model.interestRate = Param(within=NonNegativeReals)
# lifetime
model.lifeTimeTechs = Param(model.In)
model.lifeTimeStorages = Param(model.Out)


## Declaring Global Parameters ##
model.timeHorizon = Param(within=NonNegativeReals)
model.bigM = Param(within=NonNegativeReals)

#loads
model.elecLoad = Param(model.Time, initialize=elecload_init)
model.heatLoad = Param(model.Time, initialize=heatload_init)
model.loads = Param(model.Time, model.Out, initialize=loads_init)
model.solarEm = Param(model.Time, initialize=solar_init)
#model.elecLoad = Param(model.Time)
#model.heatLoad = Param(model.Time)

## Global variables
model.P = Var(model.Time, model.In, domain=NonNegativeReals)
model.Pexport = Var(model.Time, model.Out, domain=NonNegativeReals)
model.Capacities = Var(model.In, model.Out, domain=NonNegativeIntegers)
model.Ytechnologies = Var(model.In, model.Out, domain=Binary)
model.Yon = Var(model.Time, model.In, domain=Binary)
model.TotalCost = Var(domain=Reals)
model.OpCost = Var(domain=NonNegativeReals)
model.MaintCost = Var(domain=NonNegativeReals)
model.IncomeExp = Var(domain=NonNegativeReals)
model.InvCost = Var(domain=NonNegativeReals)
model.TotalCarbon = Var(domain=Reals)
model.NetPresentValueTech = Var(model.In, domain=NonNegativeReals)
model.NetPresentValueStor = Var(model.Out, domain=NonNegativeReals)
#Storage variables
model.Qin = Var(model.Time, model.Out, domain=NonNegativeReals)
model.Qout = Var(model.Time, model.Out, domain=NonNegativeReals)
model.E = Var(model.Time, model.Out, domain=NonNegativeReals)
model.StorageCap = Var(model.Out)
model.Ystorage = Var(model.Out, domain=Binary)

In [990]:
#-----------------------------------------------------------------------------#
## GLobal constraints
#-----------------------------------------------------------------------------#
def loadsBalance_rule(model, t, out):
    return (model.loads[t,out] + model.Pexport[t,out] <= (model.Qout[t,out] - model.Qin[t,out] + 
                                                        sum(model.P[t,inp]*model.cMatrix[inp,out] for inp in model.In)))
model.loadsBalance = Constraint(model.Time, model.Out, rule=loadsBalance_rule)

def capacityConst_rule(model, t, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, inp] * model.cMatrix[inp,out] <= model.Capacities[inp,out])
model.capacityConst = Constraint(model.Time, model.In, model.Out, rule=capacityConst_rule)

def maxCapacity_rule(model, disp, out):
    if disp==4 or disp==7 or disp==8 or model.cMatrix[disp,out] == 0:
        return(Constraint.Skip)
    else:
        return (model.Capacities[disp, out] <= model.maxCapTechs[disp])
model.maxCapacity = Constraint(model.DispTechs, model.Out, rule=maxCapacity_rule)

def capacity_rule(model, inp, out):
    if model.cMatrix[inp,out] <= 0:
        return(model.Capacities[inp,out] == 0)
    else:
        return(Constraint.Skip)
model.capacity_feasibility = Constraint(model.In, model.Out, rule=capacity_rule)

#def maxCapacityBIS_rule(model):
#    return (model.Capacities[2, 2] == 10)
#model.maxCapacityBIS = Constraint(rule=maxCapacityBIS_rule)

def carbonConst_rule(model):
    return (model.TotalCarbon <= model.maxCarbon)
model.carbonConst = Constraint(rule=carbonConst_rule)

In [991]:
#-----------------------------------------------------------------------------#
## Specific constraints 
#-----------------------------------------------------------------------------#
def partLoadL_rule(model, t, disp, out):  
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.partLoad[disp, out] * model.Capacities[disp, out] <= 
                                                                    (model.P[disp, out] * model.cMatrix[disp,out] 
                                                                    + model.bigM * (1 - model.Yon[t, disp])))
model.partLoadL = Constraint(model.Time, model.DispTechs, model.Out, rule=partLoadL_rule)

def partLoadU_rule(model, t, disp, out):    
    if model.cMatrix[disp,out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, disp] * model.cMatrix[disp, out] <= model.bigM * model.Yon[t, disp])
model.partLoadU = Constraint(model.Time, model.DispTechs, model.Out, rule=partLoadU_rule)

def solarInput_rule(model, t, sol, out):
    if model.cMatrix[sol, out] <= 0:
        return(Constraint.Skip)
    else:
        return (model.P[t, sol] == model.solarEm[t] * model.Capacities[sol, out])
model.solarInput = Constraint(model.Time, model.SolarTechs, model.Out, rule=solarInput_rule) 

def roofArea_rule(model):
    return (model.Capacities[5,1] + model.Capacities[6,2] <= model.maxSolarArea)
model.roofArea = Constraint(rule=roofArea_rule)

def fixCostConst_rule(model, inp, out):
    return (model.Capacities[inp,out] <= model.bigM * model.Ytechnologies[inp,out])
model.fixCostConst = Constraint(model.In, model.Out, rule=fixCostConst_rule)

def h2pRatio_rule(model, chp):
    return (model.Capacities[chp,2] == model.cMatrix[chp,2] / model.cMatrix[chp,1] * model.Capacities[chp,1])
model.h2pRatio = Constraint(model.CHP, rule=h2pRatio_rule)

def chp_rule(model, chp):
    return (model.Ytechnologies[chp,1]==model.Ytechnologies[chp,2])
model.chpConst = Constraint(model.CHP, rule=chp_rule)

def chpDiscrete_rule(model, chp):
    return (model.Capacities[chp,1] == model.maxCapTechs[chp] * model.Ytechnologies[chp,1])
model.chpDiscrete = Constraint(model.CHP, rule=chpDiscrete_rule)

In [992]:
#-----------------------------------------------------------------------------#
## Storage constraints
#-----------------------------------------------------------------------------#

def storageBalance_rule(model, t, out):
    return (model.E[t, out] == (
                                model.lossesStorStanding[out] * model.E[(t-1), out] 
                                + model.chargingEff[out] * model.Qin[t, out] 
                                - (1/model.dischLosses[out]) * model.Qout[t, out]))
model.storageBalance = Constraint(model.SubTime, model.Out, rule=storageBalance_rule)

def storageInitBattery_rule(model):
    return (model.E[1, 1] == model.StorageCap[1] * model.minSoC[1])
model.storageInitBattery = Constraint(rule=storageInitBattery_rule)

def storageInitThermal1_rule(model):
    return (model.E[1, 2] == model.E[8760, 2])
model.storageInitThermal1 = Constraint(rule=storageInitThermal1_rule)

def storageInitThermal2_rule(model):
    return (model.Qout[1, 2] == 0)
model.storageInitThermal2 = Constraint(rule=storageInitThermal2_rule)

def storageChargeRate_rule(model, t, out):
    return (model.Qin[t,out] <= model.maxStorCh[out] * model.StorageCap[out])
model.storageChargeRate = Constraint(model.Time, model.Out, rule=storageChargeRate_rule)

def storageDischRate_rule(model, t, out):
    return (model.Qout[t,out] <= model.maxStorDisch[out] * model.StorageCap[out])
model.storageDischRate = Constraint(model.Time, model.Out, rule=storageDischRate_rule)

def storageMinState_rule(model, t, out):
    return (model.E[t,out] >= model.StorageCap[out] * model.minSoC[out])
model.storageMinState = Constraint(model.Time, model.Out, rule=storageMinState_rule)

def storageCap_rule(model, t, out):
    return (model.E[t,out] <= model.StorageCap[out] )
model.storageCap = Constraint(model.Time, model.Out, rule=storageCap_rule)


In [993]:
#-----------------------------------------------------------------------------#
## Objective functions
#-----------------------------------------------------------------------------#
def objective_rule(model):
    return (model.TotalCost)
model.Total_Cost = Objective(rule=objective_rule, sense=minimize)

#def objective_rule(model):
#    return (model.TotalCarbon)
#model.Total_Carbon = Objective(rule=objective_rule, sense=minimize)

def opCost_rule(model):
    return(model.OpCost == ((sum (model.opPrices[inp] 
                            * sum(model.P[t,inp] for t in model.Time)
                            for inp in model.In)))
                            )
model.opCost = Constraint(rule=opCost_rule) 

def maintenanceCost_rule(model):
    return(model.MaintCost == (sum(model.P[t,inp] * 
                              model.cMatrix[inp,out] * model.omvCosts[inp]
                              for t in model.Time for inp in model.In for out in model.Out)
                              ))
model.maintCost = Constraint(rule=maintenanceCost_rule)

def incomeExp_rule(model):
    return(model.IncomeExp == (sum(model.feedInTariffs[out] * 
                            sum(model.Pexport[t,out] for t in model.Time) 
                            for out in model.Out))
                            )
model.incomeExp = Constraint(rule=incomeExp_rule)

def invCost_rule(model):
    return(model.InvCost == (sum(model.NetPresentValueTech[inp] * (model.linCapCosts[inp,out] * model.Capacities[inp,out] + model.fixCapCosts[inp,out] * model.Ytechnologies[inp,out]) for inp in model.W_Ogrid for out in model.Out)
                            + sum(model.NetPresentValueStor[out] * model.linStorageCosts[out] * model.StorageCap[out] for out in model.Out))
                            )
model.invCost = Constraint(rule=invCost_rule) 

def totalCost_rule(model):
    return(model.TotalCost == model.InvCost + model.OpCost + model.MaintCost - model.IncomeExp)
model.totalCost = Constraint(rule=totalCost_rule) 

def totalCarbon_rule(model):
    return(model.TotalCarbon == sum(model.carbonFactors[inp] * sum(model.P[t,inp] for t in model.Time) for inp in model.In))
model.totalCarbon = Constraint(rule=totalCarbon_rule)

# To replace npv by Capital recovery factory (inverse of net present value)
def npvTech_rule(model, inp):
    return (model.NetPresentValueTech[inp] == 1 / (((1 + model.interestRate) ** model.lifeTimeTechs[inp] - 1) / (model.interestRate * ((1 + model.interestRate) ** model.lifeTimeTechs[inp]))))
model.npvTech = Constraint(model.W_Ogrid, rule=npvTech_rule)

def npvStor_rule(model, out):
    return (model.NetPresentValueStor[out] == 1 / (((1 + model.interestRate) ** model.lifeTimeStorages[out] - 1) / (model.interestRate * ((1 + model.interestRate) ** model.lifeTimeStorages[out]))))
model.npvStor = Constraint(model.Out, rule=npvStor_rule)


In [994]:
#-----------------------------------------------------------------------------#
## Print model ##
#-----------------------------------------------------------------------------#
#model.pprint()

instance = model.create(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\generic_model.dat')
#instance.pprint()

    
opt = SolverFactory("gurobi")
solver_manager = SolverManagerFactory("serial")
results = solver_manager.solve(instance, opt=opt, tee=True,timelimit=None)
#results = opt.solve(instance)

#sends results to stdout
#results.write()
instance.load(results)

def pyomo_save_results(options=None, instance=None, results=None):
    OUTPUT = open(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\data\Results_generic_hub.txt','w')
    print(results, file=OUTPUT)
    OUTPUT.close()
#pyomo_save_results(instance=instance, results=results)    

#Example of how to print variables
print(instance.TotalCost.value)
#for i in instance.Pnaturalgas:
#    print (instance.Pnaturalgas[i], instance.Pnaturalgas[i].value)

#print("--- %s seconds ---" % (time.time() - start_time))

## create the instance
#instance = model.create(’DiseaseEstimation.dat’)
## define the solver and its options
#solver = ’ipopt’
#opt = SolverFactory( solver )
#if opt is None:
#raise ValueError, "Problem constructing solver \
#‘"+str(solver)
#opt.set_options(’max_iter=2’)
## create the solver manager
#solver_manager = SolverManagerFactory( ’serial’ )
## solve
#results = solver_manager.solve(instance, opt=opt, tee=True,\
#timelimit=None)
#instance.load(results)
## display results
#display(instance)

	Model.create_instance() to create a concrete instance from an abstract
	model.  You do not need to call Model.create() for a concrete model.
Optimize a model with 359211 rows, 184010 columns and 1068133 nonzeros
Model has 1 quadratic constraint
Variable types: 140178 continuous, 43832 integer (43816 binary)
Coefficient statistics:
  Matrix range     [1e-03, 2e+03]
  QMatrix range    [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-02, 3e+04]
Presolve removed 245340 rows and 105171 columns
Presolve time: 1.21s
Presolved: 113872 rows, 78839 columns, 323438 nonzeros
Variable types: 78835 continuous, 4 integer (0 binary)
Found heuristic solution: objective 73768.358412

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.2264000e+33   2.821454e+34   1.226400e+03     30s
   21705    3.0887265e+04   7.069397e+07   0.000000e+00     35s
   40965    4.2658558e+04   1.139857e+05   0.000000e+0

In [10]:
instance = model.create(r'C:\Users\mobo\OneDrive\0PhD\Python-based energy hub\Generic_energy_hub_PYTHON\generic_model.dat')
